In [263]:
#import the module
import json
import pandas as pd
import requests
import numpy as np
from datetime import datetime

#retrieve theMovieDB.org API key
def get_keys(path):
    '''
    Init signature: get_keys(filename=None)
    Docstring:     
    get_keys(filename='') -> dict
    Retrieve API key stored on file.  return a dictionary.
    
    Parameters
    ----------
    filename: Loation of the json file where the API key is stored.
      
    '''
    with open(path) as f:
        return json.load(f)


In [38]:
# import the api key for movieDB using the get_keys function
keys = get_keys('moviedb.json')

#assign API value to variabloe
api_key = keys['api_key']


To be deleted
https://api.themoviedb.org/3/discover/movie?api_key=da53dbf2111373cbf50f44192834054a&language=en-US
&sort_by=popularity.desc&include_adult=false&include_video=false&page=3&primary_release_date.gte=2010-01-01

In [ ]:
# Define get request parameters
url = 'https://api.themoviedb.org/3/discover/movie?'
langauge = 'en-US'
sort_by = 'desc'
include_adult = 'false'
include_video = 'false'
primary_release_date_gt = '2010-01-01'

url_param = {
                'api_key': api_key,
                'language': langauge,
                'sort_by': sort_by,
                'include_adult': include_adult,
                'include_video': include_video,
                'primary_release_date.gt': primary_release_date_gt
            }

# url_param.update({'page': 1})
# test = requests.get(url, params=url_param)
# test.json()['results']

# API only return 1 page with 20 results.  Set the loop to 500 times since the max number of pages allowed is 10,000.
for i in range(1,500): 
    # update the page number using the iterate value
    url_param.update({'page': i})
    # create the response ojb.
    resp = requests.get(url, params=url_param)
    # if request status is sucessful and the value of key ['page'] is 1
    if resp.status_code == 200 and resp.json()['page'] == 1:
        # create the DataFrame
        df = pd.DataFrame.from_dict(resp.json()['results'])
    # if request status is sucessful and able to read the value of key ['page']
    elif resp.status_code == 200 and resp.json().get('page', 10000) < 10000:
        print(resp.json()['page'])
        a = pd.DataFrame.from_dict(resp.json()['results'])
        df = pd.concat([df, a], sort=False)
    # if request status is sucessful and unable to read the value of key ['page'], assign value 10,000 and quit the loop  
    elif resp.status_code == 200 and resp.json().get('page', 10000) == 10000:
        break


In [211]:
#export tmdb movie lists from dataframe to csv
df.to_csv('tmdb.csv', index=False)

In [137]:
# reset dataframe index due to duplicates from concat of dataframe
df2 = df.reset_index()

# remove column index as well as irrelevent columns
df2.drop(['index', 'video', 'poster_path', 'backdrop_path'], axis=1, inplace=True)


In [139]:
# create a function to send request to pull movie details
# define get request parameters

def get_movie_bud_rev(movieid,rev_bud='revenue'):
    '''
    Init signature: get_keys(movie_id)
    Docstring:     
    get_movie_rev -> float
    Retrieve 
    
    Parameters
    ----------
    url: base TMDB API base URL
    api_key: TMDB api_key
    movie_id: TMDB movie id

    '''
    url = 'https://api.themoviedb.org/3/movie/' + str(movieid) + '?'
    url_param = {
                    'api_key': api_key,
                    'language': 'en-US',
                }
    resp = requests.get(url, params=url_param)
    if resp.status_code == 200:# and bud_rev == 'revenue':
        resp_bud_rev = resp.json()[rev_bud]
#         print(movieid)
    else:
        resp_bud_rev = np.nan
  
        
    return resp_bud_rev


(200, nan)

In [242]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 9 columns):
popularity           9980 non-null float64
vote_count           9980 non-null int64
vote_average         9980 non-null float64
title                9980 non-null object
release_date         9889 non-null object
original_language    9980 non-null object
revenue              9850 non-null float64
budget               9850 non-null float64
genre_names          20 non-null object
dtypes: float64(4), int64(1), object(4)
memory usage: 701.8+ KB


In [ ]:
#create the budget and the revenue columns
df2['revenue'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'revenue'))
df2['budget'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'budget'))



In [216]:
# get the genre id classification from tmdb web site
#https://api.themoviedb.org/3/genre/movie/list?api_key=da53dbf2111373cbf50f44192834054a&language=en-US
def get_genre():
    g_resp = requests.get(
        'https://api.themoviedb.org/3/genre/movie/list?api_key='
        + str(api_key) + '&language=en-US'
        )
    if g_resp.status_code == 200:
        genre_dict =  sorted(g_resp.json()['genres'], key=lambda x: x['id'])
        
    return genre_dict
        

In [217]:
def match_genre_id(gid):
    genre_name = []
    genre_dict = get_genre()
    for i in gid:
        for j in genre_dict:
            if j['id'] == i:
                genre_name.append(j['name'])
    
    return genre_name

In [218]:
# match_genre_id(df2['genre_ids'].head()[0])
df2['genre_names'] = df2['genre_ids'].head(20).apply(lambda x: match_genre_id(x)).head(209)




In [220]:
#export df2 before dropping columns id, genre_ids, adult, overview, original_title
df2.to_csv('df2_b4_drop_col.csv', index=False)

In [223]:
# drop irrelevent columns
df2.drop(['id', 'genre_ids','overview', 'adult', 'original_title'], axis=1, inplace=True)

In [281]:
df2['release_date'][300][0:4]
x: 'Good' if any([word in x.lower() for word in ['awesome', 'love', 'good', 'great']]) else 'Bad').head()

'1966'

In [303]:
df2['release_date'] = df2['release_date'].map(lambda x: x[0:4] if not pd.isnull(x) else x)


In [305]:
df2.head()

,popularity,vote_count,vote_average,title,release_date,original_language,revenue,budget,genre_names
0,0.600,1,6.0,The Strange Cohabitation,2013,ko,NaN,NaN,"[Mystery, Horror]"
1,5.511,66,7.5,A Brief History of Time,1991,en,0.0,0.0,[Documentary]
2,19.668,5053,7.3,American Psycho,2000,en,34266564.0,7000000.0,"[Thriller, Drama, Crime, Horror]"
3,15.054,1177,7.5,Frida,2002,en,56298474.0,12000000.0,"[Drama, Romance]"
4,5.485,19,5.8,The Return of the King,1980,en,0.0,0.0,"[Animation, Fantasy, Family, TV Movie]"
